# kernel

> IPythonKernel based on pystata

The latest documentation for implementing a wrapper kernel is [here](https://ipython.readthedocs.io/en/stable/development/wrapperkernels.html), but the current code deviates from those instructions somewhat, inheriting from a different class, [IPythonKernel](https://github.com/ipython/ipykernel/blob/main/ipykernel/ipkernel.py).

In [ ]:
#| default_exp kernel
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import test_eq
from fastcore.basics import patch_to

In [ ]:
#| export
from ipykernel.ipkernel import IPythonKernel
from nbstata.config import get_config, launch_stata
from nbstata import parsers
import os
import sys
from packaging import version

In [ ]:
#| export
class PyStataKernel(IPythonKernel):
    implementation = 'nbstata'
    implementation_version = '0.0.1'
    language = 'stata'
    language_version = '17'
    language_info = {
        'name': 'stata',
        'mimetype': 'text/x-stata',
        'codemirror_mode': 'stata',
        'file_extension': '.do',
    }
    banner = "nbstata: a Jupyter kernel for Stata based on pystata"

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.stata_ready = False
        self.shell.execution_count = 0
        self.echo = False
        self.noecho = False
        self.quietly = False
        self.magic_handler = None
        self.env = None

In [ ]:
#| export
@patch_to(PyStataKernel)
def init_stata(self):
    self.env = get_config()
    if self.env['echo'] not in ('True', 'False', 'None'):
        raise OSError("'" + self.env['echo'] + "' is not an acceptable value for 'echo'.")

    launch_stata(self.env['stata_dir'], self.env['edition'],
                 False if self.env['splash']=='False' else True)
    import pystata # can only be imported after adding Stata to sys.path

    self.set_graph_format(self.env['graph_format'])

    from .magics import StataMagics
    self.magic_handler = StataMagics()

    self.stata_ready = True

@patch_to(PyStataKernel)
def set_graph_format(self, graph_format):
    if graph_format == 'pystata':
        pass
    else:
        from pystata.config import set_graph_format
        set_graph_format(graph_format)

In [ ]:
#| export
@patch_to(PyStataKernel)
def do_execute(self, code, silent, store_history=True, user_expressions=None,
               allow_stdin=False):
    if not self.stata_ready:
        self.init_stata()

    # Read settings from env dict every time so that these can be modified by magics 
    # for each cell.
    if self.env['echo'] == 'None':
        self.noecho = True
        self.echo = False
    elif self.env['echo'] == 'True':
        self.noecho = False
        self.echo = True
    else:
        self.noecho = False
        self.echo = False
    self.quietly = False

    # Process magics
    code = self.magic_handler.magic(code, self)

    # Execute Stata code after magics
    if code != '':
        # Supress echo?
        if self.noecho and not self.quietly:
            code = parsers.clean_code(code,noisily=True)
            self.quietly = True
        from pystata.stata import run
        run(code, quietly=self.quietly, inline=True, echo=self.echo)

    self.shell.execution_count += 1

    return {
        'status': 'ok',
        'execution_count': self.execution_count,
        'payload': [],
        'user_expressions': {},
        }

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()